<a href="https://colab.research.google.com/github/emiliemanning/Blender-3D-Segmentation/blob/main/coco_to_mesh_disconnected_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:

!pip install pycocotools opencv-python-headless matplotlib scikit-image trimesh

import os
import json
from math import ceil
import numpy as np
from pycocotools import mask as maskUtils
from skimage.draw import polygon
from skimage.measure import marching_cubes
from scipy.ndimage import gaussian_filter
import trimesh

In [37]:
from google.colab import files
uploaded = files.upload()

Saving predicted_segmentations_coco (6).json to predicted_segmentations_coco (6) (2).json


In [38]:
from google.colab import files
import os, json

coco_json_path = list(uploaded.keys())[0]
coco_json_path = os.path.join('/content', coco_json_path)

with open(coco_json_path) as f:
    coco = json.load(f)


# first image to find height/width
if len(coco['images']) == 0:
    raise ValueError("No images found in the COCO JSON file.")

first_img = coco['images'][0]
image_height = first_img['height']
image_width = first_img['width']
num_slices = len(coco['images'])

print(f"found: height={image_height}, width={image_width}, slices={num_slices}")
output_dir = './meshes/'
os.makedirs(output_dir, exist_ok=True)

desired_blender_width = 8.0  # in Blender units
blender_unit_per_pixel = desired_blender_width / image_width
voxel_size_xy = blender_unit_per_pixel
voxel_size_z = 1 * blender_unit_per_pixel

sigma_xy = 2.5
sigma_z = 2.0

image_id_to_index = {img['id']: idx for idx, img in enumerate(sorted(coco['images'], key=lambda x: x['file_name']))}
category_id_to_name = {cat['id']: cat['name'] for cat in coco['categories']}

category_name_to_volume = {
    name: np.zeros((num_slices, image_height, image_width), dtype=np.uint8)
    for name in category_id_to_name.values()
}


found: height=350, width=500, slices=500


In [39]:
for ann in coco['annotations']:
    slice_idx = image_id_to_index[ann['image_id']]
    category_id = ann['category_id']

    if category_id not in category_id_to_name:
        print(f"Skipping annotation ID {ann['id']} due to unknown category ID: {category_id}")
        continue

    category_name = category_id_to_name[category_id]
    segmentation = ann['segmentation']

    if isinstance(segmentation, list):  #polygon format
        for seg in segmentation:
            x = np.array(seg[0::2])
            y = np.array(seg[1::2])
            rr, cc = polygon(y, x, (image_height, image_width))
            category_name_to_volume[category_name][slice_idx, rr, cc] = 1
    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        mask = maskUtils.decode(segmentation)
        category_name_to_volume[category_name][slice_idx, mask > 0] = 1
    else:
        print(f"Unknown segmentation format for annotation ID {ann['id']}")

In [40]:
# random color per part (mesh and tiff stack)
from scipy.ndimage import label
from PIL import Image

def generate_shared_color_map(volume, seed=42):
    structure = np.ones((3, 3, 3))  # 26-connectivity
    labeled_volume, num_labels = label(volume, structure=structure)

    if num_labels == 0:
        return labeled_volume, np.zeros((1, 3), dtype=np.uint8)  # Only background

    np.random.seed(seed)
    colors = np.random.randint(0, 255, size=(num_labels + 1, 3), dtype=np.uint8)  # label 0 = background
    return labeled_volume, colors


In [41]:
# @title Combines specific classes for tiff stack (eg: tum + roi)
combined_class_map = {
    'roi_tum': ['roi', 'tum'],
}
for new_class_name, component_names in combined_class_map.items():
    merged_volume = np.zeros_like(next(iter(category_name_to_volume.values())), dtype=np.uint8)
    for cname in component_names:
        if cname not in category_name_to_volume:
            print(f"Warning: class '{cname}' not found for combination '{new_class_name}'")
            continue
        merged_volume |= category_name_to_volume[cname]
    category_name_to_volume[new_class_name] = merged_volume


In [42]:
# @title tif stacks (different color for disconnected classes)

from scipy.ndimage import label
from skimage.color import label2rgb
from PIL import Image

tiff_output_dir = './tiff_stacks/'
os.makedirs(tiff_output_dir, exist_ok=True)

class_component_color_maps = {}

print("\nCREATING TIFF STACKS...")
for class_name, volume in category_name_to_volume.items():
    print(f"Processing class: {class_name}")
    if np.max(volume) == 0:
        continue

    labeled_volume, colors = generate_shared_color_map(volume)
    class_component_color_maps[class_name] = (labeled_volume, colors)

    rgb_volume = np.zeros((*labeled_volume.shape, 3), dtype=np.uint8)
    for label_id in range(1, colors.shape[0]):
        mask = labeled_volume == label_id
        rgb_volume[mask] = colors[label_id]

    frames = [Image.fromarray(rgb_volume[z]) for z in range(rgb_volume.shape[0])]
    tiff_path = os.path.join(tiff_output_dir, f"{class_name}.tiff")
    frames[0].save(tiff_path, save_all=True, append_images=frames[1:])
    print(f"Saved TIFF stack: {tiff_path}")





CREATING TIFF STACKS...
Processing class: bv
Saved TIFF stack: ./tiff_stacks/bv.tiff
Processing class: roi
Saved TIFF stack: ./tiff_stacks/roi.tiff
Processing class: tum
Saved TIFF stack: ./tiff_stacks/tum.tiff
Processing class: tum_extra
Processing class: roi_tum
Saved TIFF stack: ./tiff_stacks/roi_tum.tiff


In [43]:
# Zip the tiff_stacks folder
shutil.make_archive('tiff_stacks', 'zip', './tiff_stacks')

print("Zipped: meshes.zip and tiff_stacks.zip")
# files.download('meshes.zip')
files.download('tiff_stacks.zip')

Zipped: meshes.zip and tiff_stacks.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:

# @title Create meshes for Blender
# meshing
z_thickness_factor = ceil(voxel_size_z / voxel_size_xy)

#calculate center offsets
x_center = image_width / 2.0
y_center = image_height / 2.0
z_center = (num_slices * z_thickness_factor) / 2.0


#meshes flat at borders
z_thickness_factor = ceil(voxel_size_z / voxel_size_xy)

# for class_name, volume in category_name_to_volume.items():
#     print(f"Meshing class: {class_name}")
#     if np.max(volume) == 0:
#         print(f"Skipping {class_name} (empty)")
#         continue
#     #scale z
#     scaled_volume = np.repeat(volume, z_thickness_factor, axis=0)
#     # gaussian
#     smoothed_volume = gaussian_filter(
#         scaled_volume.astype(np.float32),
#         sigma=(sigma_z, sigma_xy, sigma_xy)
#     )

#     # Pad volume to avoid meshing artifacts at edges
#     padded_volume = np.pad(smoothed_volume, pad_width=1, mode='constant', constant_values=0)
#     #z min
#     if np.any(volume[0, :, :]):
#         mask_at_edge = volume[0, :, :]
#         padded_volume[1, 1:-1, 1:-1][mask_at_edge > 0] = 1

#     #z max
#     if np.any(volume[-1, :, :]):
#         mask_at_edge = volume[-1, :, :]
#         padded_volume[-2, 1:-1, 1:-1][mask_at_edge > 0] = 1

#     #u min
#     if np.any(volume[:, 0, :]):
#         mask_at_edge = volume[:, 0, :]
#         padded_volume[1:-1, 1, 1:-1][mask_at_edge > 0] = 1

#     #u max
#     if np.any(volume[:, -1, :]):
#         mask_at_edge = volume[:, -1, :]
#         padded_volume[1:-1, -2, 1:-1][mask_at_edge > 0] = 1

#     #x min
#     if np.any(volume[:, :, 0]):
#         mask_at_edge = volume[:, :, 0]
#         padded_volume[1:-1, 1:-1, 1][mask_at_edge > 0] = 1

#     # x max
#     if np.any(volume[:, :, -1]):
#         mask_at_edge = volume[:, :, -1]
#         padded_volume[1:-1, 1:-1, -2][mask_at_edge > 0] = 1

#     verts, faces, normals, _ = marching_cubes(padded_volume, level=0.47)

#     # scale vertices to BLENSER units
#     new_voxel_z = voxel_size_z / z_thickness_factor
#     verts_scaled = verts * [new_voxel_z, voxel_size_xy, voxel_size_xy]

#     # center
#     z_center = padded_volume.shape[0] / 2.0
#     y_center = padded_volume.shape[1] / 2.0
#     x_center = padded_volume.shape[2] /  2.0
#     verts_scaled[:, 0] -= z_center * new_voxel_z
#     verts_scaled[:, 1] -= y_center * voxel_size_xy
#     verts_scaled[:, 2] -= x_center * voxel_size_xy

#     # reorder for Blender coords
#     mesh = trimesh.Trimesh(vertices=verts_scaled[:, [2, 1, 0]], faces=faces)
#     mesh.export(os.path.join(output_dir, f"{class_name}.ply"))

# print("MESHES EXPORTED")


In [45]:
# @title disconnected meshes for each class
# for class_name, volume in category_name_to_volume.items():
#     print(f"Meshing class: {class_name}")
#     if np.max(volume) == 0:
#         continue

#     if class_name not in class_component_color_maps:
#         print(f"Skipping mesh for {class_name} — no matching TIFF color map.")
#         continue
#     labeled_volume, colors = class_component_color_maps[class_name]

#     scaled_volume = np.repeat(volume, z_thickness_factor, axis=0)
#     smoothed_volume = gaussian_filter(
#         scaled_volume.astype(np.float32),
#         sigma=(sigma_z, sigma_xy, sigma_xy)
#     )
#     padded_volume = np.pad(smoothed_volume, pad_width=1, mode='constant', constant_values=0)

#     edges = {
#         "z_min": (volume[0, :, :], (1, slice(1, -1), slice(1, -1))),
#         "z_max": (volume[-1, :, :], (-2, slice(1, -1), slice(1, -1))),
#         "y_min": (volume[:, 0, :], (slice(1, -1), 1, slice(1, -1))),
#         "y_max": (volume[:, -1, :], (slice(1, -1), -2, slice(1, -1))),
#         "x_min": (volume[:, :, 0], (slice(1, -1), slice(1, -1), 1)),
#         "x_max": (volume[:, :, -1], (slice(1, -1), slice(1, -1), -2))
#     }
#     for mask, idx in edges.values():
#         if np.any(mask):
#             padded_volume[idx][mask > 0] = 1
#     verts, faces, normals, _ = marching_cubes(padded_volume, level=0.47)

#     new_voxel_z = voxel_size_z / z_thickness_factor
#     verts_scaled = verts * [new_voxel_z, voxel_size_xy, voxel_size_xy]

#     z_center = padded_volume.shape[0] / 2.0
#     y_center = padded_volume.shape[1] / 2.0
#     x_center = padded_volume.shape[2] / 2.0
#     verts_scaled[:, 0] -= z_center * new_voxel_z
#     verts_scaled[:, 1] -= y_center * voxel_size_xy
#     verts_scaled[:, 2] -= x_center * voxel_size_xy

#     mesh = trimesh.Trimesh(vertices=verts_scaled[:, [2, 1, 0]], faces=faces)
#     submeshes = mesh.split(only_watertight=False)

#     submeshes = sorted(submeshes, key=lambda m: -len(m.faces))

#     for i, submesh in enumerate(submeshes):
#         if i + 1 >= colors.shape[0]:
#             print(f"More submeshes than labels for {class_name}, skipping extras.")
#             break
#         rgb_color = colors[i + 1]  # label 0 is background
#         color_array = np.tile(rgb_color, (submesh.vertices.shape[0], 1))
#         submesh.visual.vertex_colors = color_array

#         # Create subdirectory for the class
#         class_dir = os.path.join(output_dir, class_name)
#         os.makedirs(class_dir, exist_ok=True)

#         out_path = os.path.join(class_dir, f"{class_name}_part{i+1}.ply")
#         submesh.export(out_path)
#         print(f"Saved mesh with color: {out_path}")


In [46]:
import shutil
shutil.make_archive('meshes', 'zip', './meshes')
files.download('meshes.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
# @title multicolor single mesh per class

import os

# Set output folder
output_dir = './mesh_connected_multicolor/'
os.makedirs(output_dir, exist_ok=True)

for class_name, volume in category_name_to_volume.items():
    print(f"Meshing class: {class_name}")
    if np.max(volume) == 0:
        continue

    # Get matching labeled_volume and colors
    if class_name not in class_component_color_maps:
        print(f"Skipping mesh for {class_name} — no matching TIFF color map.")
        continue
    labeled_volume, colors = class_component_color_maps[class_name]

    # Smooth volume
    scaled_volume = np.repeat(volume, z_thickness_factor, axis=0)
    smoothed_volume = gaussian_filter(
        scaled_volume.astype(np.float32),
        sigma=(sigma_z, sigma_xy, sigma_xy)
    )
    padded_volume = np.pad(smoothed_volume, pad_width=1, mode='constant', constant_values=0)

    # Reinforce edges
    edges = {
        "z_min": (volume[0, :, :], (1, slice(1, -1), slice(1, -1))),
        "z_max": (volume[-1, :, :], (-2, slice(1, -1), slice(1, -1))),
        "y_min": (volume[:, 0, :], (slice(1, -1), 1, slice(1, -1))),
        "y_max": (volume[:, -1, :], (slice(1, -1), -2, slice(1, -1))),
        "x_min": (volume[:, :, 0], (slice(1, -1), slice(1, -1), 1)),
        "x_max": (volume[:, :, -1], (slice(1, -1), slice(1, -1), -2))
    }
    for mask, idx in edges.values():
        if np.any(mask):
            padded_volume[idx][mask > 0] = 1

    # Marching cubes
    verts, faces, normals, values = marching_cubes(padded_volume, level=0.47)

    # Scale and center
    new_voxel_z = voxel_size_z / z_thickness_factor
    verts_scaled = verts * [new_voxel_z, voxel_size_xy, voxel_size_xy]

    z_center = padded_volume.shape[0] / 2.0
    y_center = padded_volume.shape[1] / 2.0
    x_center = padded_volume.shape[2] / 2.0
    verts_scaled[:, 0] -= z_center * new_voxel_z
    verts_scaled[:, 1] -= y_center * voxel_size_xy
    verts_scaled[:, 2] -= x_center * voxel_size_xy

    # Reorder for Blender
    verts_reordered = verts_scaled[:, [2, 1, 0]]

    # Convert to mesh and split into parts
    full_mesh = trimesh.Trimesh(vertices=verts_reordered, faces=faces, process=False)
    submeshes = full_mesh.split(only_watertight=False)

    # Assign consistent colors
    all_vertices = []
    all_faces = []
    all_colors = []

    vertex_offset = 0
    for i, submesh in enumerate(submeshes):
        if i + 1 >= colors.shape[0]:
            print(f"Too many components for {class_name}; skipping extra.")
            break

        color = colors[i + 1]  # skip background
        all_vertices.append(submesh.vertices)
        all_faces.append(submesh.faces + vertex_offset)
        all_colors.append(np.tile(color, (submesh.vertices.shape[0], 1)))
        vertex_offset += submesh.vertices.shape[0]

    if not all_vertices:
        print(f"No valid components in {class_name}")
        continue

    # Combine into one mesh
    combined_vertices = np.vstack(all_vertices)
    combined_faces = np.vstack(all_faces)
    combined_colors = np.vstack(all_colors)

    # Assign vertex colors
    final_mesh = trimesh.Trimesh(vertices=combined_vertices, faces=combined_faces, process=False)
    # final_mesh.visual.vertex_colors = combined_colors
    combined_colors_float = (combined_colors / 255.0).clip(0, 1)
    final_mesh.visual.vertex_colors = combined_colors_float


    # Export to folder
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    out_path = os.path.join(class_dir, f"{class_name}.ply")
    final_mesh.export(out_path)
    print(f"Saved colored mesh: {out_path}")


Meshing class: bv
Saved colored mesh: ./mesh_connected_multicolor/bv/bv.ply
Meshing class: roi
Saved colored mesh: ./mesh_connected_multicolor/roi/roi.ply
Meshing class: tum
Too many components for tum; skipping extra.
Saved colored mesh: ./mesh_connected_multicolor/tum/tum.ply
Meshing class: tum_extra
Meshing class: roi_tum
Saved colored mesh: ./mesh_connected_multicolor/roi_tum/roi_tum.ply


In [48]:
shutil.make_archive('mesh_connected_multicolor', 'zip', './mesh_connected_multicolor')
files.download('mesh_connected_multicolor.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>